## v0.2.x Refactoring tests

This notebook contains the various tests and sense-checks done to validate the v0.2.0 refactor.

In [1]:
import numpy as np
import pandas as pd
import quadgrid as qg
from tqdm.notebook import tqdm

In [2]:
qg.__version__

'0.2.1'

## Create test datasets

### Random points

Generate random lon-lat pairs to test old vs new qid functions.

In [3]:
n = 1_000_000
np.random.seed(0)
lons = np.random.uniform(-180, 180, size=n)
lats = np.random.uniform(-90, 90, size=n)
lonlats_random = np.stack([lons, lats]).T
print(lonlats_random.shape)
lonlats_random

(1000000, 2)


array([[ 17.57286141,  34.1200881 ],
       [ 77.46817189,  85.41305698],
       [ 36.99481539,  55.45064977],
       ...,
       [-49.46712162,  77.10454385],
       [  5.94234593,  70.50490566],
       [147.09811824,  -1.46335055]])

### Grid points

Generate regular grid whose points fall on quadgrid boundaries.

In [4]:
res = 0.2
urg = qg.QuadGrid(res=res)
urg

QuadGrid(0.2°) | -180°<=lon<=180° | -90°<=lat<=90°

Extract the lon, lats in both degrees and mas, and add half a cell width:

In [5]:
lonlats_grid = np.stack([urg.lons_2d, urg.lats_2d]).T + res/2
print(lonlats_grid.shape)
lonlats_grid

(1620000, 2)


array([[-179.8,  -89.8],
       [-179.6,  -89.8],
       [-179.4,  -89.8],
       ...,
       [ 179.6,   90. ],
       [ 179.8,   90. ],
       [ 180. ,   90. ]])

## Tests

Create two `QTree` objects, one in degrees, one in mas, to test new code.

In [6]:
res = 0.2
qt = qg.QTree(res, mas=False)
qt_mas = qg.QTree(res, mas=True)

### Random points

#### Convert to qids

In [7]:
qids_random_old = np.array([qg.ll2qid(*ll, res) for ll in tqdm(lonlats_random)])
qids_random_new = np.array([qt.ll2qid(*ll) for ll in tqdm(lonlats_random)])
qids_random_new_mas = np.array([qt_mas.ll2qid(*llm) for llm in tqdm(lonlats_random)])

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

Old and new all same?

In [8]:
(qids_random_old == qids_random_new).all()

np.True_

In [9]:
(qids_random_old == qids_random_new_mas).all()

np.True_

All good for the new degrees and mas calculations.

Check that numpy-generated qids are the same as the point-by-point loop calculation:

In [10]:
qids_random_old_np = qg.lls2qids(*lonlats_random.T, res)
qids_random_new_np = qt.lls2qids(*lonlats_random.T)
qids_random_new_mas_np = qt_mas.lls2qids(*lonlats_random.T)

print((qids_random_old==qids_random_old_np).all())
print((qids_random_new==qids_random_new_np).all())
print((qids_random_new_mas==qids_random_new_mas_np).all())

True
True
True


#### Convert from qids

In [11]:
lls_random_old = np.array([qg.qid2ll(qid, res) for qid in qids_random_old])
lls_random_new = np.array([qt.qid2ll(qid) for qid in qids_random_new])
lls_random_new_mas = np.array([qt_mas.qid2ll(qid) for qid in qids_random_new_mas])

Any differences between old and new qid centroids?

In [12]:
np.allclose(lls_random_old, lls_random_new)

True

In [13]:
np.allclose(lls_random_old, lls_random_new_mas)

True

All good.

Check that numpy-generated centroids are the same as the point-by-point calculation:

In [14]:
lls_random_old_np = np.array(qg.qids2lls(qids_random_old, res)).T
lls_random_new_np = np.array(qt.qids2lls(qids_random_new)).T
lls_random_new_mas_np = np.array(qt_mas.qids2lls(qids_random_new_mas)).T

print(np.allclose(lls_random_old, lls_random_old_np))
print(np.allclose(lls_random_new, lls_random_new_np))
print(np.allclose(lls_random_new_mas, lls_random_new_mas_np))

True
True
True


### Grid points

#### Convert to qids

In [15]:
qids_grid_old = np.array([qg.ll2qid(*ll, res) for ll in tqdm(lonlats_grid)])
qids_grid_new = np.array([qt.ll2qid(*ll) for ll in tqdm(lonlats_grid)])
qids_grid_new_mas = np.array([qt_mas.ll2qid(*llm) for llm in tqdm(lonlats_grid)])

  0%|          | 0/1620000 [00:00<?, ?it/s]

  0%|          | 0/1620000 [00:00<?, ?it/s]

  0%|          | 0/1620000 [00:00<?, ?it/s]

All good.

Old and new all the same?

In [16]:
(qids_grid_old == qids_grid_new).all()

np.False_

In [17]:
(qids_grid_old == qids_grid_new_mas).all()

np.False_

In [18]:
(qids_grid_new == qids_grid_new_mas).all()

np.False_

There are differences, which is expected since the old code struggled with the edge cases for which this test has been designed. One check is to ensure that all generated qids are unique.

In [19]:
np.unique(qids_grid_old).size == qids_grid_old.size

False

In [20]:
np.unique(qids_grid_new).size == qids_grid_new.size

False

In [21]:
np.unique(qids_grid_new_mas).size == qids_grid_new_mas.size

True

This is as expected - the old and new qids are not unique despite being produced from a unique set of lon, lat coordinates, whereas the new qids calculated in mas are unique.

Check that numpy-generated qids are the same as the point-by-point calculation:

In [22]:
qids_grid_old_np = qg.lls2qids(*lonlats_grid.T, res)
qids_grid_new_np = qt.lls2qids(*lonlats_grid.T)
qids_grid_new_mas_np = qt_mas.lls2qids(*lonlats_grid.T)

print((qids_grid_old==qids_grid_old_np).all())
print((qids_grid_new==qids_grid_new_np).all())
print((qids_grid_new_mas==qids_grid_new_mas_np).all())

True
True
True


#### Convert from qids

In [23]:
lls_grid_old = np.array([qg.qid2ll(qid, res) for qid in tqdm(qids_grid_old)])
lls_grid_new = np.array([qt.qid2ll(qid) for qid in tqdm(qids_grid_new)])
lls_grid_new_mas = np.array([qt_mas.qid2ll(qid) for qid in tqdm(qids_grid_new_mas)])

  0%|          | 0/1620000 [00:00<?, ?it/s]

  0%|          | 0/1620000 [00:00<?, ?it/s]

  0%|          | 0/1620000 [00:00<?, ?it/s]

Any differences between old and new coordinates?

In [24]:
np.allclose(lls_grid_old, lls_grid_new)

False

In [25]:
np.allclose(lls_grid_old, lls_grid_new_mas)

False

In [26]:
np.allclose(lls_grid_new, lls_grid_new_mas)

False

Yes, there are differences, as expected. Since we're testing a regularly spaced grid, one sense check is to see if the spacing of the original grid coordinates is preserved in the centroids of the assigned qids.

In [27]:
np.allclose(np.diff(lonlats_grid, axis=0), np.diff(lls_grid_old, axis=0))

False

In [28]:
np.allclose(np.diff(lonlats_grid, axis=0), np.diff(lls_grid_new, axis=0))

False

In [29]:
np.allclose(np.diff(lonlats_grid, axis=0), np.diff(lls_grid_new_mas, axis=0))

True

Again, only the new grid centroid lon, lats calculated based on mas are consistent with the originally-specified grid spacing.

In [30]:
lonlats_grid

array([[-179.8,  -89.8],
       [-179.6,  -89.8],
       [-179.4,  -89.8],
       ...,
       [ 179.6,   90. ],
       [ 179.8,   90. ],
       [ 180. ,   90. ]])

In [31]:
lls_grid_new

array([[-179.9,  -89.9],
       [-179.5,  -89.9],
       [-179.5,  -89.9],
       ...,
       [ 179.5,   89.9],
       [ 179.7,   89.9],
       [ 179.9,   89.9]])

In [32]:
lls_grid_new_mas

array([[-179.9,  -89.9],
       [-179.7,  -89.9],
       [-179.5,  -89.9],
       ...,
       [ 179.5,   89.9],
       [ 179.7,   89.9],
       [ 179.9,   89.9]])

Check that numpy-generated centroids are the same as the point-by-point calculation:

In [33]:
lls_grid_old_np = np.array(qg.qids2lls(qids_grid_old, res)).T
lls_grid_new_np = np.array(qt.qids2lls(qids_grid_new)).T
lls_grid_new_mas_np = np.array(qt_mas.qids2lls(qids_grid_new_mas)).T

print(np.allclose(lls_grid_old, lls_grid_old_np))
print(np.allclose(lls_grid_new, lls_grid_new_np))
print(np.allclose(lls_grid_new_mas, lls_grid_new_mas_np))

True
True
True


All good.